<a href="https://colab.research.google.com/github/YogeshParihar1311/Deep_Learning/blob/main/gibot_task_miscellaneous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization

In [120]:
# Load the data
train_features = pd.read_csv('train.csv')
train_labels = pd.read_csv('trainLabels.csv').iloc[:9999,:]
test_features = pd.read_csv('test.csv',names=train_features.columns,header=None)

In [121]:
train_features.shape,train_labels.shape

((9999, 146), (9999, 34))

In [122]:
train_features.tail()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
9994,9995,NO,NO,jComfqYTXYSeH3GvqcOhwPmldb+BCdVJDKKDNkdtw2w=,Kr2CC15nSwDjdpyAeVh4vOuIaHuC/Q7cL9BAK28JoG8=,1.207136,0.082855,0.918960,0.313880,0.495189,...,0.0,0.81,4677,3307,YES,NO,YES,1,0.502268,0.486637
9995,9996,NO,NO,Pr5enXjzVzVjZziZxrDcgnyu6CLUftmEbnp6TctyJbU=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,1.414798,0.000000,1.000000,0.000000,0.357369,...,0.0,1.00,1262,892,YES,NO,YES,15,0.890135,0.346276
9996,9997,YES,YES,9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,1.413677,0.000000,1.000000,0.000000,0.668517,...,0.0,1.00,1261,892,YES,NO,YES,5,0.726457,0.659001
9997,9998,NO,NO,9zkXU3f6YnRPjsWi3lKSCLseIGrleg00tpRI4OplABw=,gOZBAoajyr6i7GgON0N7q5+KE4JTwH3OUM0lZOWMuG8=,1.294118,0.000000,1.000000,0.000000,0.570707,...,1.0,1.00,1188,918,NO,NO,NO,3,0.450980,0.561448
9998,9999,YES,YES,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,0.660897,0.042735,0.946581,0.086966,0.510278,...,0.0,0.88,3308,4680,YES,NO,YES,0,0.604274,0.499395


In [123]:
train_labels.tail()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
9994,9995,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9995,9996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9996,9997,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,9998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9998,9999,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [124]:
train_features.isnull().sum()

id         0
x1      1426
x2      1426
x3      1426
x4      1426
        ... 
x141       0
x142       0
x143       0
x144       0
x145       0
Length: 146, dtype: int64

In [125]:
numerical_columns = train_features.select_dtypes(include=np.number).columns.tolist()
categorical_columns = train_features.select_dtypes(exclude=np.number).columns.tolist()

In [126]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_features[numerical_columns] = imputer.fit_transform(train_features[numerical_columns])

In [127]:
for column in categorical_columns:
    mode_value = train_features[column].mode()[0]  # Calculate mode for categorical columns
    train_features[column].fillna(mode_value, inplace=True)

In [128]:
train_features.drop("id",axis=1,inplace=True)
train_labels.drop("id",axis=1,inplace=True)
test_features.drop("id",axis=1,inplace=True)

In [129]:
# train_features.dropna(inplace=True)
# test_features.dropna(inplace=True)

In [130]:
train_features.shape,train_labels.shape

((9999, 145), (9999, 33))

In [131]:
categorical_features = train_features.select_dtypes(include="O").columns

In [132]:
from sklearn.preprocessing import LabelEncoder

# Apply Label Encoding or One-Hot Encoding to categorical features
for feature in categorical_features:
    # Initialize Label Encoder or One-Hot Encoder
    encoder = LabelEncoder()  # Use LabelEncoder for ordinal, OneHotEncoder for nominal
    # encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')  # Use OneHotEncoder for nominal

    # Fit and transform the categorical feature
    # train_features[feature] = encoder.fit_transform(train_features[feature])
    # test_features[feature] = encoder.fit_transform(test_features[feature])

    frequency_map = train_features[feature].value_counts(normalize=True)
    train_features[f'{feature}_FrequencyEncoded'] = train_features[feature].map(frequency_map)
    train_features.drop(feature,axis=1,inplace=True)


    frequency_map = test_features[feature].value_counts(normalize=True)
    test_features[f'{feature}_FrequencyEncoded'] = test_features[feature].map(frequency_map)
    test_features.drop(feature,axis=1,inplace=True)

<ipython-input-132-c1dcc1febcfc>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_features[f'{feature}_FrequencyEncoded'] = train_features[feature].map(frequency_map)
<ipython-input-132-c1dcc1febcfc>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_features[f'{feature}_FrequencyEncoded'] = train_features[feature].map(frequency_map)
<ipython-input-132-c1dcc1febcfc>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [133]:
# train_features_upsampled, _ = resample(train_features, train_labels, n_samples=train_labels.shape[0], random_state=42)

In [134]:
# train_labels_downsampled = resample(train_labels, n_samples=7402, replace=False, random_state=42)

In [135]:
train_features.shape

(9999, 145)

In [136]:
train_features.head()

,x5,x6,x7,x8,x9,x15,x16,x17,x18,x19,...,x116_FrequencyEncoded,x117_FrequencyEncoded,x126_FrequencyEncoded,x127_FrequencyEncoded,x128_FrequencyEncoded,x129_FrequencyEncoded,x130_FrequencyEncoded,x140_FrequencyEncoded,x141_FrequencyEncoded,x142_FrequencyEncoded
0,0.576561,0.073139,0.481394,0.115697,0.472474,42.0,0.396065,3.0,6.0,0.991018,...,0.991999,0.275628,0.551055,0.993999,0.555256,0.974897,0.781478,0.525853,0.978398,0.533653
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.991999,0.724372,0.551055,0.993999,0.555256,0.974897,0.781478,0.525853,0.978398,0.533653
2,1.341803,0.051422,0.935572,0.041440,0.501710,2.0,0.838475,3.0,5.0,0.966122,...,0.991999,0.724372,0.448945,0.993999,0.444744,0.974897,0.781478,0.474147,0.978398,0.466347
3,0.653912,0.041471,0.940787,0.090851,0.556564,37.0,0.127405,8.0,15.0,0.959171,...,0.991999,0.275628,0.551055,0.993999,0.555256,0.974897,0.781478,0.474147,0.978398,0.533653
4,1.415919,0.000000,1.000000,0.000000,0.375297,1.0,0.523543,4.0,11.0,0.963004,...,0.991999,0.724372,0.448945,0.993999,0.555256,0.974897,0.781478,0.474147,0.978398,0.466347


In [137]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

In [138]:
# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
test_features_scaled = scaler.transform(test_features)

In [139]:
X_train_scaled.shape,y_train.shape

((7999, 145), (7999, 33))

In [140]:
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.datasets import make_multilabel_classification
# from sklearn.model_selection import train_test_split

# # Generating sample data (Replace this with your actual data)
# # X, y = make_multilabel_classification(n_samples=5000, n_features=146, n_classes=34, n_labels=4)

# # # Splitting the data into train and test sets
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# # Define the neural network model
# model = Sequential()

# model.add(Dense(128, input_dim=145, activation='relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())

# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))
# model.add(BatchNormalization())

# model.add(Dense(33, activation='sigmoid'))  # Output layer with 34 nodes for 34 classes

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Define early stopping criteria
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# # Train the model
# model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.1,callbacks=[early_stopping])

# # Evaluate the model
# # loss, accuracy = model.evaluate(X_test, y_test)
# # print(f"Test Accuracy: {accuracy:.2f}")


In [141]:
!pip install keras-tuner

In [142]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# Generate sample data (Replace this with your actual data)
# X, y = make_multilabel_classification(n_samples=5000, n_features=146, n_classes=34, n_labels=4)

# Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Define the model building function
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units1', min_value=64, max_value=256, step=32), input_dim=145, activation='relu'))
    model.add(Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(BatchNormalization())
    model.add(Dense(33, activation='sigmoid'))  # Output layer with 34 nodes for 34 classes

    optimizer_choice = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    if optimizer_choice == 'adam':
        optimizer = optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    elif optimizer_choice == 'rmsprop':
        optimizer = optimizers.RMSprop(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))
    else:
        optimizer = optimizers.SGD(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG'))

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Instantiate the tuner
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=10, directory='my_dir', project_name='multi_label')

# Perform hyperparameter search
tuner.search(X_train_scaled, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(patience=3)])

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Optimal Hyperparameters: {best_hps}")

# Retrain the model with best hyperparameters
model = tuner.hypermodel.build(best_hps)
model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.1, callbacks=[EarlyStopping(patience=5)])


Reloading Tuner from my_dir/multi_label/tuner0.json
Optimal Hyperparameters: <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7d87921a80a0>
Epoch 1/100
225/225 [==============================] - 2s 4ms/step - loss: 0.1234 - accuracy: 0.5979 - val_loss: 0.0542 - val_accuracy: 0.7050
Epoch 2/100
225/225 [==============================] - 1s 3ms/step - loss: 0.0569 - accuracy: 0.6768 - val_loss: 0.0456 - val_accuracy: 0.7475
Epoch 3/100
225/225 [==============================] - 1s 3ms/step - loss: 0.0517 - accuracy: 0.6916 - val_loss: 0.0441 - val_accuracy: 0.7325
Epoch 4/100
225/225 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.7104 - val_loss: 0.0401 - val_accuracy: 0.7700
Epoch 5/100
225/225 [==============================] - 1s 3ms/step - loss: 0.0474 - accuracy: 0.7177 - val_loss: 0.0383 - val_accuracy: 0.7350
Epoch 6/100
225/225 [==============================] - 1s 3ms/step - loss: 0.0458 - accuracy: 0.7233 - val_los

In [143]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val_scaled, y_val)
print(f"Test Accuracy: {accuracy:.2f}")

63/63 [==============================] - 0s 3ms/step - loss: 0.0392 - accuracy: 0.7575
Test Accuracy: 0.76


In [144]:
X_val_scaled.shape

(2000, 145)

In [145]:
test_features.shape

(2000, 145)

In [146]:
y_pred = model.predict(test_features).round().astype(int)

63/63 [==============================] - 0s 1ms/step


In [147]:
y_pred[4]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
output = []
for m,n in enumerate(y_pred):
  for i,j in enumerate(n):
    output.append([f"{1700000+m+1}_y{i+1}",j])

In [ ]:
result = pd.DataFrame(output,columns=["id_label","pred"])
result

,id_label,pred
0,1700001_y1,0
1,1700001_y2,0
2,1700001_y3,0
3,1700001_y4,0
4,1700001_y5,0
...,...,...
65995,1702000_y29,0
65996,1702000_y30,0
65997,1702000_y31,0
65998,1702000_y32,0


In [ ]:
# result.to_csv("result.csv",index=None)